## Parameter Selection

This script runs SMD and FMD for different n values to compare the solving time (Figure 3).

In [ ]:
import sys

# Add the path to the FairnessML package to the system path
sys.path.append(r'..\..\src\Parameter_Selection')
sys.path.append(r'..\..\src\RBTree') 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import gurobipy as gp
from gurobipy import GRB
from statistics import mean
from tqdm import tqdm
from PS_test_functions import n_num_test

In [ ]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
parent_path2 = os.path.dirname(parent_path)

"""
Parameter Setting for the Experiment. Modify the below listed parameters for the experiment.
    n_list_nt: list of n values to run the experiment. This should be the main parameter you may change for the experiment.
    K_nt: per-iteration sample size K
    J_nt: number of cohorts
    L_nt: number of treatments
    repeats_nt: number of repeats for each n
"""


J_nt = 5  # Number of Cohorts
L_nt = 10 # Number of Treatments

# n_list_nt = np.linspace(5000,10000,2)
n_list_nt = [3000,5000]

seed_nt = 1234
np.random.seed(seed_nt)
#n_list_nt = np.linspace(10000,20000,11)
m_nt = 3  # constraint number for figure 1
repeats_nt = 1
K_nt = 100
print_opt = 1

SMD_run_time_arr = np.zeros(repeats_nt)
FMD_run_time_arr = np.zeros(repeats_nt)
SMD_real_T_arr = np.zeros(repeats_nt)
FMD_real_T_arr = np.zeros(repeats_nt)
SMD_T_arr = np.zeros(repeats_nt)
FMD_T_arr = np.zeros(repeats_nt)
SMD_run_dual_time_arr = np.zeros(repeats_nt)
FMD_run_dual_time_arr = np.zeros(repeats_nt)
for n_num in n_list_nt:
    for rep_idx in range(repeats_nt):
        stat_SMD, stat_FMD, _ = n_num_test(int(n_num), J_nt, L_nt, m_nt, K_nt, print_opt = 1, feas_opt = 1)
        SMD_run_time_arr[rep_idx] = stat_SMD.total_solved_time
        FMD_run_time_arr[rep_idx] = stat_FMD.total_solved_time
        SMD_real_T_arr[rep_idx] = stat_SMD.real_T_list[0]
        FMD_real_T_arr[rep_idx] = stat_FMD.real_T_list[0]
        SMD_T_arr[rep_idx] = stat_SMD.max_iter
        FMD_T_arr[rep_idx] = stat_FMD.max_iter
        SMD_run_dual_time_arr[rep_idx] = stat_SMD.dual_gap_time
        FMD_run_dual_time_arr[rep_idx] = stat_FMD.dual_gap_time
        data_arr = np.vstack(
            (SMD_run_time_arr, SMD_real_T_arr, SMD_T_arr, FMD_run_time_arr, FMD_real_T_arr,
             FMD_T_arr,SMD_run_dual_time_arr,FMD_run_dual_time_arr)).T
        df = pd.DataFrame(data_arr, columns=['SMD_Solve_time', 'SMD_Iter', 'SMD_Max_Iter', \
                                             'FMD_Solve_time', 'FMD_Iter', 'FMD_Max_Iter',
                                             'SMD_Dual_Solve_time','FMD_Dual_Solve_time'])

        custom_path = '/results/Parameter_Selection/n_num_result/n={}_J={}_L={}_m={}_K={}.csv'.format(n_num, J_nt,
                                                                                       L_nt, m_nt, K_nt)
        save_path = parent_path2 + custom_path
        """
        df contains the following columns:
            Columns Regarding SMD:
                SMD_Solve_time: Time to solve the SMD model
                SMD_Iter: Number of iterations to solve the SMD model
                SMD_Max_Iter: Maximum number of iterations to solve the SMD model
                SMD_Dual_Solve_time: Total time Computing Duality gap for SMD
            Columns Regarding FMD:
                FMD_Solve_time: Time to solve the FMD model
                FMD_Iter: Number of iterations to solve the FMD model
                FMD_Max_Iter: Maximum number of iterations to solve the FMD model
                FMD_Dual_Solve_time: Total time Computing Duality gap for FMD
        """
        df.to_csv(save_path, index=False)




